# Mechanism of action (MoA) of drug prediction using classification algorithms
#### First experince - Multi-label techniques with classification models

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import random
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score,recall_score
import sklearn.metrics as metrics

# **select featuers**

In [ ]:

train = pd.read_csv("../input/lish-moa/train_features.csv",index_col = 0)

train['cp_type'] = train['cp_type'].map({'trt_cp':1.0, 'ctl_vehicle':0.0})
train['cp_dose'] = train['cp_dose'].map({'D1':0.0, 'D2':1.0})
train['cp_time'] = train['cp_time'].map({24:0.0, 48: .5 , 72:1.0})


#select featuers  by correlation matrix  
train_f = pd.read_csv('../input/lish-moa/train_features.csv')
test_f = pd.read_csv('../input/lish-moa/test_features.csv')

train_f['dataset'] = 'train_f'
test_f['dataset'] = 'test_f'
df = pd.concat([train_f, test_f])

#split the features into type starting with -g and -c
train_columns = train_f.columns.to_list()
g_list = [i for i in train_columns if i.startswith('g-')]
c_list = [i for i in train_columns if i.startswith('c-')]
rest=['cp_type','cp_time','cp_dose']


#combine columns and select random features
columns = g_list + c_list+rest
for_correlation = list(set([columns[np.random.randint(0, len(columns)-1)] for i in range(200)]))[:40]
data = df[for_correlation]



#then loop over this feature and test every pair of the correlation have a value more than 0.6 choose it.
cols = ['cp_time'] + columns
all_columns = []
for i in range(0, len(cols)):
    for j in range(i+1, len(cols)):
        if abs(train[cols[i]].corr(train[cols[j]])) < 0.0003:
            all_columns = all_columns + [cols[i], cols[j]]

all_columns = list(set(all_columns))
print('Number of relevant features:', len(all_columns))

best_features = df[all_columns]
print('relevant features:\n',best_features.columns)


In [ ]:
train_F=train[best_features.columns]

In [ ]:
train_F.shape

In [ ]:
train_F['cp_type']

In [ ]:
train_F['cp_type'].value_counts()

In [ ]:
d = pd.read_csv('../input/lish-moa/train_targets_scored.csv',index_col=0)

In [ ]:
inhibitors = [col for col in d.columns if 'inhibitor' in col]
activators = [col for col in d.columns if 'activator' in col]
antagonists = [col for col in d.columns if 'antagonist' in col]
agonists = [col for col in d.columns if 'agonist' in col]
modulators = [col for col in d.columns if 'modulator' in col]
receptors = [col for col in d.columns if 'receptor' in col]
receptors_ago = [col for col in d.columns if 'receptor_agonist' in col]
receptors_anta = [col for col in d.columns if 'receptor_antagonist' in col]

In [ ]:
print('inhibitors',len(inhibitors))
print('activators',len(activators))
print('antagonists',len(antagonists))
print('agonists',len(agonists))
print('modulators',len(modulators))
print('receptors',len(receptors))
print('receptors_ago',len(receptors_ago))
print('receptors_anta',len(receptors_anta))

In [ ]:
d['receptors_anta']=d['acetylcholine_receptor_antagonist']|d['adenosine_receptor_antagonist']|d['adrenergic_receptor_antagonist']|d['androgen_receptor_antagonist']|d['angiotensin_receptor_antagonist']|d['cannabinoid_receptor_antagonist']|d['cc_chemokine_receptor_antagonist']|d['cck_receptor_antagonist']|d['cholinergic_receptor_antagonist']|d['dopamine_receptor_antagonist']|d['estrogen_receptor_antagonist']|d['gaba_receptor_antagonist']|d['glutamate_receptor_antagonist']|d['histamine_receptor_antagonist']|d['leukotriene_receptor_antagonist']|d['mineralocorticoid_receptor_antagonist']|d['neuropeptide_receptor_antagonist']|d['opioid_receptor_antagonist']|d['orexin_receptor_antagonist']|d['ppar_receptor_antagonist']|d['progesterone_receptor_antagonist']|d['prostanoid_receptor_antagonist']|d['retinoid_receptor_antagonist']|d['serotonin_receptor_antagonist']|d['sigma_receptor_antagonist']|d['smoothened_receptor_antagonist']

In [ ]:
d['receptors_ago']=d['acetylcholine_receptor_agonist']|d['adenosine_receptor_agonist']|d['adrenergic_receptor_agonist']|d['androgen_receptor_agonist']|d['benzodiazepine_receptor_agonist']|d['cannabinoid_receptor_agonist']|d['dopamine_receptor_agonist']|d['estrogen_receptor_agonist']|d['fatty_acid_receptor_agonist']|d['gaba_receptor_agonist']|d['glucocorticoid_receptor_agonist']|d['glutamate_receptor_agonist']|d['gonadotropin_receptor_agonist']|d['histamine_receptor_agonist']|d['imidazoline_receptor_agonist']|d['nicotinic_receptor_agonist']|d['opioid_receptor_agonist']|d['ppar_receptor_agonist']|d['progesterone_receptor_agonist']|d['retinoid_receptor_agonist']|d['serotonin_receptor_agonist']|d['sigma_receptor_agonist']|d['sphingosine_receptor_agonist']|d['vitamin_d_receptor_agonist']

In [ ]:
d['receptors']=d['acetylcholine_receptor_agonist']|d['acetylcholine_receptor_antagonist']|d['adenosine_receptor_agonist']|d['adenosine_receptor_antagonist']|d['adrenergic_receptor_agonist']|d['adrenergic_receptor_antagonist']|d['androgen_receptor_agonist']|d['androgen_receptor_antagonist']|d['angiotensin_receptor_antagonist']|d['benzodiazepine_receptor_agonist']|d['cannabinoid_receptor_agonist']|d['cannabinoid_receptor_antagonist']|d['cc_chemokine_receptor_antagonist']|d['cck_receptor_antagonist']|d['cholinergic_receptor_antagonist']|d['dopamine_receptor_agonist']|d['dopamine_receptor_antagonist']|d['estrogen_receptor_agonist']|d['estrogen_receptor_antagonist']|d['fatty_acid_receptor_agonist']|d['gaba_receptor_agonist']|d['gaba_receptor_antagonist']|d['glucocorticoid_receptor_agonist']|d['glutamate_receptor_agonist']|d['glutamate_receptor_antagonist']|d['gonadotropin_receptor_agonist']|d['histamine_receptor_agonist']|d['histamine_receptor_antagonist']|d['imidazoline_receptor_agonist']|d['leukotriene_receptor_antagonist']|d['mineralocorticoid_receptor_antagonist']|d['neuropeptide_receptor_antagonist']|d['nicotinic_receptor_agonist']|d['opioid_receptor_agonist']|d['opioid_receptor_antagonist']|d['orexin_receptor_antagonist']|d['ppar_receptor_agonist']|d['ppar_receptor_antagonist']|d['progesterone_receptor_agonist']|d['progesterone_receptor_antagonist']|d['prostanoid_receptor_antagonist']|d['retinoid_receptor_agonist']|d['retinoid_receptor_antagonist']|d['serotonin_receptor_agonist']|d['serotonin_receptor_antagonist']|d['sigma_receptor_agonist']|d['sigma_receptor_antagonist']|d['smoothened_receptor_antagonist']|d['sphingosine_receptor_agonist']|d['tgf-beta_receptor_inhibitor']|d['transient_receptor_potential_channel_antagonist']|d['tropomyosin_receptor_kinase_inhibitor']|d['vitamin_d_receptor_agonist']

In [ ]:
d['agonists']= d['acetylcholine_receptor_agonist']|d['acetylcholine_receptor_antagonist']|d['adenosine_receptor_agonist']|d['adenosine_receptor_antagonist']|d['adrenergic_receptor_agonist']|d['adrenergic_receptor_antagonist']|d['androgen_receptor_agonist']|d['androgen_receptor_antagonist']|d['angiotensin_receptor_antagonist']|d['atp-sensitive_potassium_channel_antagonist']|d['benzodiazepine_receptor_agonist']|d['cannabinoid_receptor_agonist']|d['cannabinoid_receptor_antagonist']|d['cc_chemokine_receptor_antagonist']|d['cck_receptor_antagonist']|d['cholinergic_receptor_antagonist']|d['corticosteroid_agonist']|d['dopamine_receptor_agonist']|d['dopamine_receptor_antagonist']|d['estrogen_receptor_agonist']|d['estrogen_receptor_antagonist']|d['fatty_acid_receptor_agonist']|d['gaba_receptor_agonist']|d['gaba_receptor_antagonist']|d['glucocorticoid_receptor_agonist']|d['glutamate_receptor_agonist']|d['glutamate_receptor_antagonist']|d['gonadotropin_receptor_agonist']|d['histamine_receptor_agonist']|d['histamine_receptor_antagonist']|d['imidazoline_receptor_agonist']|d['leukotriene_receptor_antagonist']|d['lxr_agonist']|d['mineralocorticoid_receptor_antagonist']|d['neuropeptide_receptor_antagonist']|d['nicotinic_receptor_agonist']|d['opioid_receptor_agonist']|d['opioid_receptor_antagonist']|d['orexin_receptor_antagonist']|d['potassium_channel_antagonist']|d['ppar_receptor_agonist']|d['ppar_receptor_antagonist']|d['progesterone_receptor_agonist']|d['progesterone_receptor_antagonist']|d['prostanoid_receptor_antagonist']|d['retinoid_receptor_agonist']|d['retinoid_receptor_antagonist']|d['serotonin_receptor_agonist']|d['serotonin_receptor_antagonist']|d['sigma_receptor_agonist']|d['sigma_receptor_antagonist']|d['smoothened_receptor_antagonist']|d['sphingosine_receptor_agonist']|d['tachykinin_antagonist']|d['tlr_agonist']|d['transient_receptor_potential_channel_antagonist']|d[  'vitamin_d_receptor_agonist']|d['vitamin_d_receptor_agonist']|d['tlr_antagonist']|d['trpv_agonist']|d['trpv_antagonist']|d['trpv_antagonist']

In [ ]:
d['antagonists']=d['acetylcholine_receptor_antagonist']|d['adenosine_receptor_antagonist']|d['adrenergic_receptor_antagonist']|d['androgen_receptor_antagonist']|d['angiotensin_receptor_antagonist']|d['atp-sensitive_potassium_channel_antagonist']|d['cannabinoid_receptor_antagonist']|d['cc_chemokine_receptor_antagonist']|d['cck_receptor_antagonist']|d['cholinergic_receptor_antagonist']|d['dopamine_receptor_antagonist']|d['estrogen_receptor_antagonist']|d['gaba_receptor_antagonist']|d['glutamate_receptor_antagonist']|d['histamine_receptor_antagonist']|d['leukotriene_receptor_antagonist']|d['mineralocorticoid_receptor_antagonist']|d['neuropeptide_receptor_antagonist']|d['opioid_receptor_antagonist']|d['orexin_receptor_antagonist']|d['potassium_channel_antagonist']|d['ppar_receptor_antagonist']|d['progesterone_receptor_antagonist']|d['prostanoid_receptor_antagonist']|d['retinoid_receptor_antagonist']|d['serotonin_receptor_antagonist']|d['sigma_receptor_antagonist']|d['smoothened_receptor_antagonist']|d['smoothened_receptor_antagonist']|d['tachykinin_antagonist']|d['tlr_antagonist']|d['transient_receptor_potential_channel_antagonist']|d['trpv_antagonist']

In [ ]:
d['activators']=d['adenylyl_cyclase_activator']|d['ampk_activator']|d['caspase_activator']|d['nrf2_activator']|d['potassium_channel_activator']

In [ ]:
d['inhibitors']=d['5-alpha_reductase_inhibitor'] |d['nitric_oxide_production_inhibitor'] |d['prostaglandin_inhibitor']|d['raf_inhibitor'] | d['ras_gtpase_inhibitor'] |d['ribonucleoside_reductase_inhibitor'] |d['thymidylate_synthase_inhibitor'] |d['tnf_inhibitor']| d['topoisomerase_inhibitor'] |d['tropomyosin_receptor_kinase_inhibitor']  | d['tubulin_inhibitor']| d['vegfr_inhibitor'] | d['thrombin_inhibitor']| d['rho_associated_kinase_inhibitor']| d['ubiquitin_specific_protease_inhibitor'] | d['protein_tyrosine_kinase_inhibitor']  |d['src_inhibitor']| d['syk_inhibitor'] |d['tgf-beta_receptor_inhibitor'] | d['tyrosine_kinase_inhibitor'] |d['sodium_channel_inhibitor'] |d['protein_phosphatase_inhibitor'] | d['rna_polymerase_inhibitor']| d['serotonin_reuptake_inhibitor'] | d['proteasome_inhibitor']| d['protein_kinase_inhibitor'] | d['pdgfr_inhibitor']| d['pdk_inhibitor'] | d['pkc_inhibitor'] | d['acat_inhibitor']| d['11-beta-hsd1_inhibitor'] |d['acetylcholinesterase_inhibitor']| d['5-alpha_reductase_inhibitor']   | d['aldehyde_dehydrogenase_inhibitor']    |d['5-alpha_reductase_inhibitor']  | d['akt_inhibitor']| d['11-beta-hsd1_inhibitor']   |d['alk_inhibitor'] | d['aromatase_inhibitor'] | d['atm_kinase_inhibitor'] |d['aurora_kinase_inhibitor'] | d['autotaxin_inhibitor']|d['bacterial_30s_ribosomal_subunit_inhibitor']|d['bacterial_50s_ribosomal_subunit_inhibitor']|d['bacterial_dna_gyrase_inhibitor'] |d['bacterial_dna_inhibitor'] | d['bacterial_membrane_integrity_inhibitor'] | d['bacterial_cell_wall_synthesis_inhibitor']  | d['atpase_inhibitor']| d['atr_kinase_inhibitor'] |d['atp_synthase_inhibitor']| d['bcl_inhibitor']|d['casein_kinase_inhibitor']|d['cholesterol_inhibitor']|d['dipeptidyl_peptidase_inhibitor']| d['gsk_inhibitor']|d['hiv_inhibitor']|d['monoacylglycerol_lipase_inhibitor'] |d['nitric_oxide_synthase_inhibitor']|d['phosphodiesterase_inhibitor']  |d['monopolar_spindle_1_kinase_inhibitor'] | d['mtor_inhibitor']| d['nfkb_inhibitor']| d['phospholipase_inhibitor']| d['pi3k_inhibitor'] | d['monoamine_oxidase_inhibitor'] | d['norepinephrine_reuptake_inhibitor']| d['p38_mapk_inhibitor']  | d['p-glycoprotein_inhibitor']| d['parp_inhibitor'] | d['mek_inhibitor']| d['membrane_integrity_inhibitor'] | d['leukotriene_inhibitor']| d['lipase_inhibitor'] | d['lipoxygenase_inhibitor'] |d['mdm_inhibitor'] |d['ikk_inhibitor'] | d['integrin_inhibitor']| d['jak_inhibitor'] |d['kit_inhibitor'] | d['hmgcr_inhibitor']| d['hsp_inhibitor'] | d['igf-1_inhibitor']  | d['histone_lysine_demethylase_inhibitor']| d['histone_lysine_methyltransferase_inhibitor'] | d['hcv_inhibitor'] |d['hdac_inhibitor'] | d['fungal_squalene_epoxidase_inhibitor']| d['gamma_secretase_inhibitor'] |d['glutamate_inhibitor']  | d['fgfr_inhibitor'] |d['flt3_inhibitor'] |d['focal_adhesion_kinase_inhibitor']  | d['dna_inhibitor'] |d['egfr_inhibitor'] | d['dihydrofolate_reductase_inhibitor']| d['cytochrome_p450_inhibitor'] |d ['coagulation_factor_inhibitor']| d['cyclooxygenase_inhibitor'] | d['catechol_o_methyltransferase_inhibitor']| d['cdk_inhibitor'] | d['chk_inhibitor'] |d['btk_inhibitor'] |d['calcineurin_inhibitor']| d['carbonic_anhydrase_inhibitor']  |d['bcr-abl_inhibitor'] | d['beta_amyloid_inhibitor']|d['bromodomain_inhibitor']  

In [ ]:
y=d.loc[:,'receptors_anta':]

In [ ]:
y

# K-nearest neighbor (KNN) algorithm

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
X_train, X_test, y_train, y_test = train_test_split(train, y,shuffle=True,test_size=0.2)

# GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
g_p={'n_neighbors':[2,4,5,6,9],
     'weights':["uniform", "distance"],
     'metric':['euclidean','manhattan','cosine'] }
gs=GridSearchCV(KNeighborsClassifier(),
                g_p,
                cv=10)
grid=gs.fit(X_train, y_train)
print (grid.best_score_)
print (grid.best_params_)
print (grid.best_estimator_)

# KNN +ClassifierChain

In [ ]:
# using classifier chains
from skmultilearn.problem_transform import ClassifierChain
# initialize classifier chains multi-label classifier
classifier = ClassifierChain(KNeighborsClassifier(n_neighbors=6, metric='cosine',weights= 'uniform'))
# Training logistic regression model on train data
classifier.fit(X_train, y_train)
# predict
predictions = classifier.predict(X_test)
# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions))
print("\n")


y_predict = classifier.predict(X_test)

print('classification report :\n',classification_report(y_test, y_predict))

br_f1=metrics.f1_score(y_test, predictions, average='micro')
br_hamm=metrics.hamming_loss(y_test,predictions)
print('Binary Relevance F1-score:',round(br_f1,3))
print('Binary Relevance Hamming Loss:',round(br_hamm,3))


# KNN +BinaryRelevance

In [ ]:
# using binary relevance
from skmultilearn.problem_transform import BinaryRelevance


# initialize binary relevance multi-label classifier
# with a gaussian naive bayes base classifier
classifier = BinaryRelevance(KNeighborsClassifier(n_neighbors=6, metric='cosine',weights= 'uniform'))

# train
classifier.fit(X_train, y_train)

# predict
y_predict = classifier.predict(X_test)

print('Test accuracy is {}'.format(accuracy_score(y_test, y_predict)))
print("\n")
print('classification report: \n',classification_report(y_test, y_predict))


br_f1=metrics.f1_score(y_test, y_predict, average='micro')
br_hamm=metrics.hamming_loss(y_test,y_predict)
print('Binary Relevance F1-score:',round(br_f1,3))
print('Binary Relevance Hamming Loss:',round(br_hamm,3))

# DT + ClassifierChain

In [ ]:
# using classifier chains..DT
from sklearn.tree import DecisionTreeClassifier
from skmultilearn.problem_transform import ClassifierChain
clf = ClassifierChain(DecisionTreeClassifier(criterion="entropy",max_depth=3,min_samples_leaf=2,random_state = 5))

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

#Accuracy vs classification report
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
# Model Accuracy, how often is the classifier correct?
print('classification report :\n',classification_report(y_test, y_pred))
br_hamm=metrics.hamming_loss(y_test,y_pred)
print('Binary Relevance Hamming Loss:',round(br_hamm,3))

# DT + BinaryRelevance

In [ ]:
# using binary relevance
from skmultilearn.problem_transform import BinaryRelevance
# Using pipeline for applying logistic regression and one vs rest classifier
clf_b = BinaryRelevance(DecisionTreeClassifier(criterion="entropy",max_depth=3,min_samples_leaf=2,random_state = 5))
# Training logistic regression model on train data
clf_b.fit(X_train,y_train)
y_pred = clf.predict(X_test)
#Accuracy vs classification report
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
# Model Accuracy, how often is the classifier correct?
print('classification report :\n',classification_report(y_test, y_pred))
br_hamm=metrics.hamming_loss(y_test,y_pred)
print('Binary Relevance Hamming Loss:',round(br_hamm,3))

# RF+ClassifierChain

In [ ]:
# using classifier chains..RFC
from sklearn.ensemble import RandomForestClassifier
classifier = ClassifierChain(RandomForestClassifier(n_estimators=100, random_state=5))

# train
classifier.fit(X_train, y_train)

# predict
y_pred = classifier.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
# Model Accuracy, how often is the classifier correct?
print('classification report :\n',classification_report(y_test, y_pred))
br_hamm=metrics.hamming_loss(y_test,y_pred)
print('Binary Relevance Hamming Loss:',round(br_hamm,3))

# RF+ BinaryRelevance

In [ ]:
# using binary relevance
from sklearn.ensemble import RandomForestClassifier
classifier = BinaryRelevance(RandomForestClassifier(n_estimators=100, random_state=5))

# train
classifier.fit(X_train, y_train)

# predict
y_pred = classifier.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
# Model Accuracy, how often is the classifier correct?
print('classification report :\n',classification_report(y_test, y_pred))
br_hamm=metrics.hamming_loss(y_test,y_pred)
print('Binary Relevance Hamming Loss:',round(br_hamm,3))

# SVM + classifier chain 

In [ ]:
from sklearn.multioutput import ClassifierChain
# initialize classifier chains multi-label classifier
classifier_chain = ClassifierChain(SVC())


# Training SVM model on train data
classifier_chain.fit(X_train, y_train)

# predict
predictions_chain = classifier_chain.predict(X_test)

# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions_chain))
print("\n")

#print the confusion matrix
print(confusion_matrix(y_test.values.argmax(axis=1), predictions_chain.argmax(axis=1)))


#the precision of the model 
from sklearn.metrics import precision_score
precision = precision_score(y_test, predictions_chain,average='macro')
print('Precision: %.3f' % precision)

# calculates recall
from sklearn.metrics import recall_score
recall = recall_score(y_test, predictions_chain, average='macro')
print('Recall: %.3f' % recall)


from sklearn.metrics import f1_score
f1_score = f1_score(y_test, predictions_chain, average='macro')
print('F-Measure: %.3f' % f1_score)

from sklearn.metrics import hamming_loss
hamming_loss(y_test, predictions_chain)

# SVM + Binary relevance  

In [ ]:
from skmultilearn.problem_transform import BinaryRelevance
clf_binary = BinaryRelevance(classifier = SVC(),require_dense = [False, True])


# train
clf_binary.fit(X_train, y_train)

# predict
predictions = clf_binary.predict(X_test)

print("Accuracy = ",accuracy_score(y_test,predictions))
print("\n")

#print the confusion matrix
print(confusion_matrix(y_test.values.argmax(axis=1), predictions.argmax(axis=1)))

#the precision of the model 
precision = precision_score(y_test, predictions,average='macro')
print('Precision: %.3f' % precision)

# calculates recall
recall = recall_score(y_test, predictions, average='macro')
print('Recall: %.3f' % recall)

f1_score = f1_score(y_test, predictions, average='macro')
print('F-Measure: %.3f' % f1_score)

hamming_loss(y_test, predictions)

# Experiment the classifiers with best features

In [ ]:
train_F=train[best_features.columns]

In [ ]:
train_F.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_F, y,shuffle=True,test_size=0.2)
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(X_train)
x_test = sc.fit_transform(X_test)

# KNN +ClassifierChain

In [ ]:
# using classifier chains
from skmultilearn.problem_transform import ClassifierChain
# initialize classifier chains multi-label classifier
classifier = ClassifierChain(KNeighborsClassifier(n_neighbors=6, metric='cosine',weights= 'uniform'))
# Training logistic regression model on train data
classifier.fit(X_train, y_train)
# predict
predictions = classifier.predict(X_test)
# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions))
print("\n")


#y_predict = classifier.predict(X_test)

print('classification report :\n',classification_report(y_test, predictions))

br_f1=metrics.f1_score(y_test, predictions, average='micro')
br_hamm=metrics.hamming_loss(y_test,predictions)
print('Binary Relevance F1-score:',round(br_f1,3))
print('Binary Relevance Hamming Loss:',round(br_hamm,3))


# KNN +BinaryRelevance

In [ ]:
# using binary relevance
from skmultilearn.problem_transform import BinaryRelevance


# initialize binary relevance multi-label classifier
# with a gaussian naive bayes base classifier
classifier = BinaryRelevance(KNeighborsClassifier(n_neighbors=6, metric='cosine',weights= 'uniform'))

# train
classifier.fit(X_train, y_train)

# predict
y_predict = classifier.predict(X_test)

print('Test accuracy is {}'.format(accuracy_score(y_test, y_predict)))
print("\n")
print('classification report: \n',classification_report(y_test, y_predict))


br_f1=metrics.f1_score(y_test, y_predict, average='micro')
br_hamm=metrics.hamming_loss(y_test,y_predict)
print('Binary Relevance F1-score:',round(br_f1,3))
print('Binary Relevance Hamming Loss:',round(br_hamm,3))

# DT + ClassifierChain

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from skmultilearn.problem_transform import ClassifierChain
clf = ClassifierChain(DecisionTreeClassifier(criterion="entropy",max_depth=3,min_samples_leaf=2,random_state = 5))

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)
#Accuracy vs classification report
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
# Model Accuracy, how often is the classifier correct?
print('classification report :\n',classification_report(y_test, y_pred))
br_hamm=metrics.hamming_loss(y_test,y_pred)
print('Binary Relevance Hamming Loss:',round(br_hamm,3))

# DT+BinaryRelevance

In [ ]:
# using binary relevance
from skmultilearn.problem_transform import BinaryRelevance
clf_b = BinaryRelevance(DecisionTreeClassifier(criterion="entropy",max_depth=3,min_samples_leaf=2,random_state = 5))
clf_b.fit(X_train,y_train)
y_pred = clf.predict(X_test)
#Accuracy vs classification report
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
# Model Accuracy, how often is the classifier correct?
print('classification report :\n',classification_report(y_test, y_pred))
br_hamm=metrics.hamming_loss(y_test,y_pred)
print('Binary Relevance Hamming Loss:',round(br_hamm,3))

# RF + ClassifierChain

In [ ]:
# using classifier chains..RFC
from sklearn.ensemble import RandomForestClassifier
classifier = ClassifierChain(RandomForestClassifier(n_estimators=100, random_state=5))

# train
classifier.fit(X_train, y_train)

# predict
y_pred = classifier.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
# Model Accuracy, how often is the classifier correct?
print('classification report :\n',classification_report(y_test, y_pred))

#br_f1=metrics.f1_score(y_test, y_pred, average='micro')
br_hamm=metrics.hamming_loss(y_test,y_pred)

#print('Binary Relevance F1-score:',round(br_f1,3))
print('Binary Relevance Hamming Loss:',round(br_hamm,3))

# RF + BinaryRelevance 

In [ ]:
# using binary relevance
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.ensemble import RandomForestClassifier
classifier = BinaryRelevance(RandomForestClassifier(n_estimators=100, random_state=5))

# train
classifier.fit(X_train, y_train)

# predict
y_pred = classifier.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
# Model Accuracy, how often is the classifier correct?
print('classification report :\n',classification_report(y_test, y_pred))

#br_f1=metrics.f1_score(y_test, y_pred, average='micro')
br_hamm=metrics.hamming_loss(y_test,y_pred)

#print('Binary Relevance F1-score:',round(br_f1,3))
print('Binary Relevance Hamming Loss:',round(br_hamm,3))

# SVM + classifier chain 

In [ ]:
# initialize classifier chains multi-label classifier
classifier_chain_best = ClassifierChain(SVC())

# Training SVM model on train data
classifier_chain_best.fit(x_train2, y_train)

# predict
predictions_chain_best = classifier_chain_best.predict(x_test2)

# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions_chain_best))
print("\n")

#print the confusion matrix
print(confusion_matrix(y_test.values.argmax(axis=1), predictions_chain_best.argmax(axis=1)))

#the precision of the model 
precision = precision_score(y_test, predictions_chain_best,average='macro')
print('Precision: %.3f' % precision)

f1_score = f1_score(y_test, predictions_chain_best, average='macro')
print('F-Measure: %.3f' % f1_score)

hamming_loss(y_test, predictions_chain_best)

# calculates recall
recall = recall_score(y_test, predictions_chain_best, average='macro')
print('Recall: %.3f' % recall)

# SVM + Binary relevance 

In [ ]:
clf_binary_best = BinaryRelevance(classifier = SVC(),require_dense = [False, True])

# train
clf_binary_best.fit(x_train2, y_train)

# predict
predictions_binary_best = clf_binary.predict(x_test2)

print("Accuracy = ",accuracy_score(y_test,predictions_binary_best))
print("\n")

#print the confusion matrix
print(confusion_matrix(y_test.values.argmax(axis=1), predictions_binary_best.argmax(axis=1)))

#the precision of the model 
precision = precision_score(y_test, predictions_binary_best,average='macro')
print('Precision: %.3f' % precision)

# calculates recall
from sklearn.metrics import recall_score
recall = recall_score(y_test, predictions_binary_best, average='macro')
print('Recall: %.3f' % recall)

from sklearn.metrics import f1_score
f1_score = f1_score(y_test, predictions_binary_best, average='macro')
print('F-Measure: %.3f' % f1_score)